In [ ]:
"""
Mon application modifiée pour l'anniversaire
"""

import pygame
import sys
import os
import random
import datetime
import moviepy
from moviepy import VideoFileClip  # Importer moviepy pour lire la vidéo

# Hardcoded metadata values
app_name = "Joyeux Anniversaire!"

# Constantes
WHITE = (255, 255, 255)
SCREEN_WIDTH, SCREEN_HEIGHT = 400, 600
BIRD_X = 50
BIRD_Y = 300
GRAVITY = 0.25
FLAP_STRENGTH = -5
PIPE_SPEED_INIT = -4
PIPE_GAP = 150
PIPE_GAP_MIN = 100  # Valeur minimale de l'écart
PIPE_GAP_MAX = 180  # Valeur maximale de l'écart
SPEED_INCREASE_INTERVAL = 3000
SCORE_INTERVAL_FOR_SPEED_INCREASE = 5

FONT_NAME = pygame.font.match_font('arial')

# **Supprimer la variable 'today' puisque nous n'en avons plus besoin**
# today = datetime.datetime.now().day

# Fonctions utilitaires
def draw_text(surf, text, size, x, y, color=(255, 255, 255), font_name=FONT_NAME):
    font = pygame.font.Font(font_name, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    text_rect.midtop = (x, y)
    surf.blit(text_surface, text_rect)

def create_snowflakes(num_flakes):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT),
             random.randint(2, 5)] for _ in range(num_flakes)]

def update_and_draw_snowflakes(screen, snowflakes):
    for flake in snowflakes:
        flake[1] += flake[2]
        pygame.draw.circle(screen, WHITE, (flake[0], flake[1]), flake[2])
        if flake[1] > SCREEN_HEIGHT:
            flake[0] = random.randint(0, SCREEN_WIDTH)
            flake[1] = -5
            flake[2] = random.randint(2, 5)

def draw_button(surf, text, x, y, width, height):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    button_clicked = False

    if x + width > mouse[0] > x and y + height > mouse[1] > y:
        pygame.draw.rect(surf, (200, 200, 200), (x, y, width, height))
        if click[0] == 1:
            button_clicked = True
    else:
        pygame.draw.rect(surf, (170, 170, 170), (x, y, width, height))

    draw_text(surf, text, 20, x + width / 2, y + 10)
    return button_clicked

def start_screen(screen):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)
    intro_text = [
        "Bienvenue au jeu d'anniversaire!",
        "Aidez le personnage à atteindre",
        "le score de 10 pour une surprise spéciale!",
        "",
        "Utilisez la flèche du haut pour monter,",
        "et celle du bas pour descendre.",
        "Bonne chance!"
    ]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)

        draw_text(screen, "Joyeux Anniversaire!", 40, SCREEN_WIDTH / 2,
                  SCREEN_HEIGHT / 10, color=(255, 50, 0)) 
        y_offset = 150
        for line in intro_text:
            draw_text(screen, line, 20, SCREEN_WIDTH / 2, y_offset)
            y_offset += 30

        if draw_button(screen, "Commencer", 150, 450, 100, 50):
            break

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def game_over_screen(screen, final_score):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)  # Dessiner les flocons de neige

        # Redessiner le texte "Game Over" et le score final dans la boucle
        draw_text(screen, "Game Over", 48, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 4)
        score_text = f"Score Final: {final_score}"
        draw_text(screen, score_text, 36, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50)

        if draw_button(screen, "Rejouer", 100, 350, 100, 50):
            return True  # Indique de recommencer le jeu
        if draw_button(screen, "Quitter", 200, 350, 100, 50):
            return False  # Indique de quitter le jeu

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def create_stars(num_stars):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT)] for _ in range(num_stars)]

def draw_stars(screen, stars):
    for star in stars:
        pygame.draw.circle(screen, (255, 255, 0), star, random.randint(1, 3))

def wait_for_key():
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                waiting = False

# **Ajouter une fonction pour jouer la vidéo**
def play_video(screen, video_path):
    clip = VideoFileClip(video_path)
    # clip = moviepy.video.fx.all.resize(clip, height=SCREEN_HEIGHT, width=SCREEN_WIDTH)  # Ajustez la hauteur de la vidéo à l'écran
    clip = clip.resized(height=SCREEN_HEIGHT, width=SCREEN_WIDTH)
    clock = pygame.time.Clock()

    for frame in clip.iter_frames(fps=30, dtype="uint8"):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        frame_surface = pygame.surfarray.make_surface(frame.swapaxes(0, 1))
        screen.blit(frame_surface, (0, 0))
        pygame.display.update()
        clock.tick(30)
    clip.close()

def draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img):
    """
    Dessiner une colonne de briques pour représenter un tuyau.

    Parameters:
    - screen: Surface pygame.
    - pipe_x: Position en x du tuyau.
    - pipe_y: Position en y de la partie supérieure du tuyau (partie inférieure de l'écran).
    - pipe_gap: Écart entre les deux parties du tuyau.
    - brique_img: Image de la brique à utiliser pour dessiner les tuyaux.
    """
    brique_height = brique_img.get_height()
    brique_width = brique_img.get_width()

    # Dessiner les briques pour la partie supérieure (tuyau inversé)
    y_top = pipe_y - pipe_gap - brique_height
    while y_top > -brique_height:
        screen.blit(pygame.transform.flip(brique_img, False, True), (pipe_x, y_top))
        y_top -= brique_height

    # Dessiner les briques pour la partie inférieure
    y_bottom = pipe_y
    while y_bottom < SCREEN_HEIGHT:
        screen.blit(brique_img, (pipe_x, y_bottom))
        y_bottom += brique_height

        
def main():
    pygame.init()
    pygame.display.set_caption(app_name)
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    start_screen(screen)

    # Initialisation de la logique du jeu
    bird_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/mario_volant.png').convert_alpha()
    bird_img = pygame.transform.scale(bird_img, (50, 50))
    pipe_img = pygame.Surface((50, 300))
    pipe_img.fill((255, 0, 0))

    stars = create_stars(10)  # Choisissez le nombre d'étoiles

    # **Supprimer la variable 'level_message_displayed'**
    # level_message_displayed = False

    # **Ajouter une variable pour vérifier si la vidéo a été jouée**
    video_played = False

    # Déclaration de la variable pour suivre le dernier niveau de score où la vitesse a été augmentée
    last_speed_increase_score = 0

    running = True
    while running:
        bird_y = BIRD_Y
        bird_velocity = 0
        pipes = [[300, random.randint(100, 400), False, random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)]]
        score = 0
        pipe_speed = PIPE_SPEED_INIT
        snowflakes = create_snowflakes(25)

        while running:
            # Logique du jeu
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        bird_velocity = FLAP_STRENGTH
                    elif event.key == pygame.K_DOWN:
                        bird_velocity = -FLAP_STRENGTH

            bird_velocity += GRAVITY
            bird_y += bird_velocity

            screen.fill((15, 5, 107))
            draw_stars(screen, stars)  # Dessiner les étoiles
            update_and_draw_snowflakes(screen, snowflakes)
            screen.blit(bird_img, (BIRD_X, bird_y))

            # Gestion des tuyaux
            for pipe in pipes:
                pipe[0] += pipe_speed
                pipe_x, pipe_y, pipe_passed, pipe_gap = pipe

                screen.blit(pipe_img, (pipe_x, pipe_y))  # Partie inférieure
                screen.blit(pygame.transform.flip(pipe_img, False, True),
                            (pipe_x, pipe_y - pipe_gap - 300))  # Partie supérieure

                # Générer de nouveaux tuyaux
                if pipe[0] < -50:
                    new_gap = random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)  # Nouvel écart aléatoire
                    new_pipe_y = random.randint(100, 400)
                    pipes.append([400, new_pipe_y, False, new_gap])
                    pipes.pop(0)

                # Vérifier si le tuyau a été passé pour augmenter le score
                if pipe[0] + pipe_img.get_width() < BIRD_X and not pipe_passed:
                    score += 1
                    pipe[2] = True  # Marquer le tuyau comme passé

            draw_text(screen, f"Score: {score}", 24, SCREEN_WIDTH - 100, 50)

            # Vérifier si le score a atteint un nouveau multiple de SCORE_INTERVAL_FOR_SPEED_INCREASE
            if score // SCORE_INTERVAL_FOR_SPEED_INCREASE > last_speed_increase_score:
                pipe_speed -= 1  # Augmenter la vitesse des tuyaux
                last_speed_increase_score = score // SCORE_INTERVAL_FOR_SPEED_INCREASE

            # **Ajouter la condition pour jouer la vidéo lorsque le score atteint 10**
            if score >= 1 and not video_played:
                play_video(screen, '/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/test_video.avi')  # Spécifiez le chemin vers votre vidéo
                video_played = True  # Éviter de rejouer la vidéo
                # Vous pouvez ajouter une pause ou un message après la vidéo si vous le souhaitez

            bird_rect = bird_img.get_rect(topleft=(BIRD_X, bird_y))
            for pipe in pipes:
                pipe_rect = pipe_img.get_rect(topleft=(pipe[0], pipe[1]))
                pipe_rect_top = pipe_img.get_rect(topleft=(pipe[0], pipe[1] - pipe[3] - 300))
                if (bird_rect.colliderect(pipe_rect) or bird_rect.colliderect(pipe_rect_top) or
                        bird_y >= SCREEN_HEIGHT - 30 or bird_y <= 0):
                    running = False
                    break

            pygame.display.update()
            pygame.time.Clock().tick(30)

        if not game_over_screen(screen, score):
            break  # Quitter si l'utilisateur choisit de ne pas recommencer

        running = True  # Recommencer le jeu
        video_played = False  # Réinitialiser pour le prochain jeu

    pygame.quit()

if __name__ == "__main__":
    main()

SystemExit: 

In [3]:
"""
Mon application modifiée pour l'anniversaire
"""

import pygame
import sys
import os
import random
import datetime
import moviepy
from moviepy import VideoFileClip  # Importer moviepy pour lire la vidéo

# Hardcoded metadata values
app_name = "Joyeux Anniversaire!"

# Constantes
WHITE = (255, 255, 255)
SCREEN_WIDTH, SCREEN_HEIGHT = 400, 600
BIRD_X = 50
BIRD_Y = 300
GRAVITY = 0.25
FLAP_STRENGTH = -5
PIPE_SPEED_INIT = -4
PIPE_GAP = 150
PIPE_GAP_MIN = 100  # Valeur minimale de l'écart
PIPE_GAP_MAX = 180  # Valeur maximale de l'écart
SPEED_INCREASE_INTERVAL = 3000
SCORE_INTERVAL_FOR_SPEED_INCREASE = 5

FONT_NAME = pygame.font.match_font('arial')

# **Supprimer la variable 'today' puisque nous n'en avons plus besoin**
# today = datetime.datetime.now().day

# Fonctions utilitaires
def draw_text(surf, text, size, x, y, color=(255, 255, 255), font_name=FONT_NAME):
    font = pygame.font.Font(font_name, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    text_rect.midtop = (x, y)
    surf.blit(text_surface, text_rect)

def create_snowflakes(num_flakes):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT),
             random.randint(2, 5)] for _ in range(num_flakes)]

def update_and_draw_snowflakes(screen, snowflakes):
    for flake in snowflakes:
        flake[1] += flake[2]
        pygame.draw.circle(screen, WHITE, (flake[0], flake[1]), flake[2])
        if flake[1] > SCREEN_HEIGHT:
            flake[0] = random.randint(0, SCREEN_WIDTH)
            flake[1] = -5
            flake[2] = random.randint(2, 5)

def draw_button(surf, text, x, y, width, height):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    button_clicked = False

    if x + width > mouse[0] > x and y + height > mouse[1] > y:
        pygame.draw.rect(surf, (200, 200, 200), (x, y, width, height))
        if click[0] == 1:
            button_clicked = True
    else:
        pygame.draw.rect(surf, (170, 170, 170), (x, y, width, height))

    draw_text(surf, text, 20, x + width / 2, y + 10)
    return button_clicked

def start_screen(screen):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)
    intro_text = [
        "Bienvenue au jeu d'anniversaire!",
        "Aidez le personnage à atteindre",
        "le score de 10 pour une surprise spéciale!",
        "",
        "Utilisez la flèche du haut pour monter,",
        "et celle du bas pour descendre.",
        "Bonne chance!"
    ]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)

        draw_text(screen, "Joyeux Anniversaire!", 40, SCREEN_WIDTH / 2,
                  SCREEN_HEIGHT / 10, color=(255, 50, 0)) 
        y_offset = 150
        for line in intro_text:
            draw_text(screen, line, 20, SCREEN_WIDTH / 2, y_offset)
            y_offset += 30

        if draw_button(screen, "Commencer", 150, 450, 100, 50):
            break

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def game_over_screen(screen, final_score):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)  # Dessiner les flocons de neige

        # Redessiner le texte "Game Over" et le score final dans la boucle
        draw_text(screen, "Game Over", 48, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 4)
        score_text = f"Score Final: {final_score}"
        draw_text(screen, score_text, 36, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50)

        if draw_button(screen, "Rejouer", 100, 350, 100, 50):
            return True  # Indique de recommencer le jeu
        if draw_button(screen, "Quitter", 200, 350, 100, 50):
            return False  # Indique de quitter le jeu

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def create_stars(num_stars):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT)] for _ in range(num_stars)]

def draw_stars(screen, stars):
    for star in stars:
        pygame.draw.circle(screen, (255, 255, 0), star, random.randint(1, 3))

def wait_for_key():
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                waiting = False

# **Ajouter une fonction pour jouer la vidéo**
def play_video(screen, video_path):
    clip = VideoFileClip(video_path)
    # clip = moviepy.video.fx.all.resize(clip, height=SCREEN_HEIGHT, width=SCREEN_WIDTH)  # Ajustez la hauteur de la vidéo à l'écran
    clip = clip.resized(height=SCREEN_HEIGHT, width=SCREEN_WIDTH)
    clock = pygame.time.Clock()

    for frame in clip.iter_frames(fps=30, dtype="uint8"):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        frame_surface = pygame.surfarray.make_surface(frame.swapaxes(0, 1))
        screen.blit(frame_surface, (0, 0))
        pygame.display.update()
        clock.tick(30)
    clip.close()

def draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img):
    """
    Dessiner une colonne de briques pour représenter un tuyau.

    Parameters:
    - screen: Surface pygame.
    - pipe_x: Position en x du tuyau.
    - pipe_y: Position en y de la partie supérieure du tuyau (partie inférieure de l'écran).
    - pipe_gap: Écart entre les deux parties du tuyau.
    - brique_img: Image de la brique à utiliser pour dessiner les tuyaux.
    """
    brique_height = brique_img.get_height()
    brique_width = brique_img.get_width()

    # Dessiner les briques pour la partie supérieure (tuyau inversé)
    y_top = pipe_y - pipe_gap - brique_height
    while y_top > -brique_height:
        screen.blit(pygame.transform.flip(brique_img, False, True), (pipe_x, y_top))
        y_top -= brique_height

    # Dessiner les briques pour la partie inférieure
    y_bottom = pipe_y
    while y_bottom < SCREEN_HEIGHT:
        screen.blit(brique_img, (pipe_x, y_bottom))
        y_bottom += brique_height

        
def main():
    pygame.init()
    pygame.display.set_caption(app_name)
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    start_screen(screen)

    # Initialisation de la logique du jeu
    bird_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/mario_volant.png').convert_alpha()
    bird_img = pygame.transform.scale(bird_img, (50, 50))
    brique_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/brique.png').convert_alpha()
    brique_img = pygame.transform.scale(brique_img, (50, 50))  # Redimensionner à 50x50 si nécessaire

    stars = create_stars(10)  # Choisissez le nombre d'étoiles

    video_played = False
    last_speed_increase_score = 0

    running = True
    while running:
        bird_y = BIRD_Y
        bird_velocity = 0
        pipes = [[300, random.randint(100, 400), False, random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)]]
        score = 0
        pipe_speed = PIPE_SPEED_INIT
        snowflakes = create_snowflakes(25)

        while running:
            # Logique du jeu
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        bird_velocity = FLAP_STRENGTH
                    elif event.key == pygame.K_DOWN:
                        bird_velocity = -FLAP_STRENGTH

            bird_velocity += GRAVITY
            bird_y += bird_velocity

            screen.fill((15, 5, 107))
            draw_stars(screen, stars)  # Dessiner les étoiles
            update_and_draw_snowflakes(screen, snowflakes)
            screen.blit(bird_img, (BIRD_X, bird_y))

            # Gestion des tuyaux
            for pipe in pipes:
                pipe[0] += pipe_speed
                pipe_x, pipe_y, pipe_passed, pipe_gap = pipe

                draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img)

                # Générer de nouveaux tuyaux
                if pipe[0] < -50:
                    new_gap = random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)  # Nouvel écart aléatoire
                    new_pipe_y = random.randint(100, 400)
                    pipes.append([400, new_pipe_y, False, new_gap])
                    pipes.pop(0)

                # Vérifier si le tuyau a été passé pour augmenter le score
                if pipe[0] + brique_img.get_width() < BIRD_X and not pipe_passed:
                    score += 1
                    pipe[2] = True  # Marquer le tuyau comme passé

            draw_text(screen, f"Score: {score}", 24, SCREEN_WIDTH - 100, 50)

            # Vérifier si le score a atteint un nouveau multiple de SCORE_INTERVAL_FOR_SPEED_INCREASE
            if score // SCORE_INTERVAL_FOR_SPEED_INCREASE > last_speed_increase_score:
                pipe_speed -= 1  # Augmenter la vitesse des tuyaux
                last_speed_increase_score = score // SCORE_INTERVAL_FOR_SPEED_INCREASE

            # **Ajouter la condition pour jouer la vidéo lorsque le score atteint 10**
            if score >= 10 and not video_played:
                play_video(screen, '/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/test_video.avi')
                video_played = True  # Éviter de rejouer la vidéo

            bird_rect = bird_img.get_rect(topleft=(BIRD_X, bird_y))
            for pipe in pipes:
                pipe_rect = pygame.Rect(pipe[0], pipe[1], brique_img.get_width(), SCREEN_HEIGHT - pipe[1])
                pipe_rect_top = pygame.Rect(pipe[0], 0, brique_img.get_width(), pipe[1] - pipe[3])
                if bird_rect.colliderect(pipe_rect) or bird_rect.colliderect(pipe_rect_top) or bird_y >= SCREEN_HEIGHT - 30 or bird_y <= 0:
                    running = False
                    break

            pygame.display.update()
            pygame.time.Clock().tick(30)

        if not game_over_screen(screen, score):
            break  # Quitter si l'utilisateur choisit de ne pas recommencer

        running = True  # Recommencer le jeu
        video_played = False  # Réinitialiser pour le prochain jeu

    pygame.quit()

if __name__ == "__main__":
    main()

SystemExit: 

In [ ]:
"""
Mon application modifiée pour l'anniversaire --> version qui fonctionne mais image plante qui vibre et qui ne compte pas pour perdre
"""

import pygame
import sys
import os
import random
import datetime
import moviepy
from moviepy import VideoFileClip  # Importer moviepy pour lire la vidéo

# Hardcoded metadata values
app_name = "Joyeux Anniversaire!"

# Constantes
WHITE = (255, 255, 255)
SCREEN_WIDTH, SCREEN_HEIGHT = 400, 600
BIRD_X = 50
BIRD_Y = 300
GRAVITY = 0.25
FLAP_STRENGTH = -5
PIPE_SPEED_INIT = -4
PIPE_GAP = 150
PIPE_GAP_MIN = 100  # Valeur minimale de l'écart
PIPE_GAP_MAX = 180  # Valeur maximale de l'écart
SPEED_INCREASE_INTERVAL = 3000
SCORE_INTERVAL_FOR_SPEED_INCREASE = 5

FONT_NAME = pygame.font.match_font('arial')

# **Supprimer la variable 'today' puisque nous n'en avons plus besoin**
# today = datetime.datetime.now().day

# Fonctions utilitaires
def draw_text(surf, text, size, x, y, color=(255, 255, 255), font_name=FONT_NAME):
    font = pygame.font.Font(font_name, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    text_rect.midtop = (x, y)
    surf.blit(text_surface, text_rect)

def create_snowflakes(num_flakes):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT),
             random.randint(2, 5)] for _ in range(num_flakes)]

def update_and_draw_snowflakes(screen, snowflakes):
    for flake in snowflakes:
        flake[1] += flake[2]
        pygame.draw.circle(screen, WHITE, (flake[0], flake[1]), flake[2])
        if flake[1] > SCREEN_HEIGHT:
            flake[0] = random.randint(0, SCREEN_WIDTH)
            flake[1] = -5
            flake[2] = random.randint(2, 5)

def draw_button(surf, text, x, y, width, height):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    button_clicked = False

    if x + width > mouse[0] > x and y + height > mouse[1] > y:
        pygame.draw.rect(surf, (200, 200, 200), (x, y, width, height))
        if click[0] == 1:
            button_clicked = True
    else:
        pygame.draw.rect(surf, (170, 170, 170), (x, y, width, height))

    draw_text(surf, text, 20, x + width / 2, y + 10)
    return button_clicked

def start_screen(screen):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)
    intro_text = [
        "Bienvenue au jeu d'anniversaire!",
        "Aidez le personnage à atteindre",
        "le score de 10 pour une surprise spéciale!",
        "",
        "Utilisez la flèche du haut pour monter,",
        "et celle du bas pour descendre.",
        "Bonne chance!"
    ]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)

        draw_text(screen, "Joyeux Anniversaire!", 40, SCREEN_WIDTH / 2,
                  SCREEN_HEIGHT / 10, color=(255, 50, 0)) 
        y_offset = 150
        for line in intro_text:
            draw_text(screen, line, 20, SCREEN_WIDTH / 2, y_offset)
            y_offset += 30

        if draw_button(screen, "Commencer", 150, 450, 100, 50):
            break

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def game_over_screen(screen, final_score):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)  # Dessiner les flocons de neige

        # Redessiner le texte "Game Over" et le score final dans la boucle
        draw_text(screen, "Game Over", 48, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 4)
        score_text = f"Score Final: {final_score}"
        draw_text(screen, score_text, 36, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50)

        if draw_button(screen, "Rejouer", 100, 350, 100, 50):
            return True  # Indique de recommencer le jeu
        if draw_button(screen, "Quitter", 200, 350, 100, 50):
            return False  # Indique de quitter le jeu

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def create_stars(num_stars):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT)] for _ in range(num_stars)]

def draw_stars(screen, stars):
    for star in stars:
        pygame.draw.circle(screen, (255, 255, 0), star, random.randint(1, 3))

def wait_for_key():
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                waiting = False

# **Ajouter une fonction pour jouer la vidéo**
def play_video(screen, video_path):
    clip = VideoFileClip(video_path)
    # clip = moviepy.video.fx.all.resize(clip, height=SCREEN_HEIGHT, width=SCREEN_WIDTH)  # Ajustez la hauteur de la vidéo à l'écran
    clip = clip.resized(height=SCREEN_HEIGHT, width=SCREEN_WIDTH)
    clock = pygame.time.Clock()

    for frame in clip.iter_frames(fps=30, dtype="uint8"):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        frame_surface = pygame.surfarray.make_surface(frame.swapaxes(0, 1))
        screen.blit(frame_surface, (0, 0))
        pygame.display.update()
        clock.tick(30)
    clip.close()

def draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img, plante_img):
    """
    Dessiner une colonne de briques pour représenter un tuyau.

    Parameters:
    - screen: Surface pygame.
    - pipe_x: Position en x du tuyau.
    - pipe_y: Position en y de la partie supérieure du tuyau (partie inférieure de l'écran).
    - pipe_gap: Écart entre les deux parties du tuyau.
    - brique_img: Image de la brique à utiliser pour dessiner les tuyaux.
    - plante_img: Image de la plante à ajouter parfois en haut du tuyau inférieur.
    """
    brique_height = brique_img.get_height()
    brique_width = brique_img.get_width()
    plante_height = plante_img.get_height()
    plante_width = plante_img.get_width()

    # Dessiner les briques pour la partie supérieure (tuyau inversé)
    y_top = pipe_y - pipe_gap - brique_height
    while y_top > -brique_height:
        screen.blit(pygame.transform.flip(brique_img, False, True), (pipe_x, y_top))
        y_top -= brique_height

    # Dessiner les briques pour la partie inférieure
    y_bottom = pipe_y
    while y_bottom < SCREEN_HEIGHT:
        screen.blit(brique_img, (pipe_x, y_bottom))
        y_bottom += brique_height

    # **Placer une plante sur le haut de la partie basse (tuyau inférieur)**
    if random.randint(0, 1) == 0:  # Probabilité à 50% pour tester
        print("Plante ajoutée sur la partie basse !")  # Debug
        screen.blit(plante_img, (pipe_x + (brique_width - plante_width) // 2, pipe_y - plante_height))
        
def main():
    pygame.init()
    pygame.display.set_caption(app_name)
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    start_screen(screen)

    # Initialisation de la logique du jeu
    bird_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/mario_volant.png').convert_alpha()
    bird_img = pygame.transform.scale(bird_img, (50, 50))
    brique_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/brique.png').convert_alpha()
    brique_img = pygame.transform.scale(brique_img, (50, 50))  # Redimensionner à 50x50 si nécessaire
    plante_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/plante.png').convert_alpha()
    plante_img = pygame.transform.scale(plante_img, (50, 75))  # Ajustez la taille selon vos besoins

    stars = create_stars(10)  # Choisissez le nombre d'étoiles

    video_played = False
    last_speed_increase_score = 0

    running = True
    while running:
        bird_y = BIRD_Y
        bird_velocity = 0
        pipes = [[300, random.randint(100, 400), False, random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)]]
        score = 0
        pipe_speed = PIPE_SPEED_INIT
        snowflakes = create_snowflakes(25)

        while running:
            # Logique du jeu
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        bird_velocity = FLAP_STRENGTH
                    elif event.key == pygame.K_DOWN:
                        bird_velocity = -FLAP_STRENGTH

            bird_velocity += GRAVITY
            bird_y += bird_velocity

            screen.fill((15, 5, 107))
            draw_stars(screen, stars)  # Dessiner les étoiles
            update_and_draw_snowflakes(screen, snowflakes)
            screen.blit(bird_img, (BIRD_X, bird_y))

            # Gestion des tuyaux
            for pipe in pipes:
                pipe[0] += pipe_speed
                pipe_x, pipe_y, pipe_passed, pipe_gap = pipe

                draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img, plante_img)

                # Générer de nouveaux tuyaux
                if pipe[0] < -50:
                    new_gap = random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)  # Nouvel écart aléatoire
                    new_pipe_y = random.randint(100, 400)
                    pipes.append([400, new_pipe_y, False, new_gap])
                    pipes.pop(0)

                # Vérifier si le tuyau a été passé pour augmenter le score
                if pipe[0] + brique_img.get_width() < BIRD_X and not pipe_passed:
                    score += 1
                    pipe[2] = True  # Marquer le tuyau comme passé

            draw_text(screen, f"Score: {score}", 24, SCREEN_WIDTH - 100, 50)

            # Vérifier si le score a atteint un nouveau multiple de SCORE_INTERVAL_FOR_SPEED_INCREASE
            if score // SCORE_INTERVAL_FOR_SPEED_INCREASE > last_speed_increase_score:
                pipe_speed -= 1  # Augmenter la vitesse des tuyaux
                last_speed_increase_score = score // SCORE_INTERVAL_FOR_SPEED_INCREASE

            # **Ajouter la condition pour jouer la vidéo lorsque le score atteint 10**
            if score >= 10 and not video_played:
                play_video(screen, '/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/test_video.avi')
                video_played = True  # Éviter de rejouer la vidéo

            bird_rect = bird_img.get_rect(topleft=(BIRD_X, bird_y))
            for pipe in pipes:
                pipe_rect = pygame.Rect(pipe[0], pipe[1], brique_img.get_width(), SCREEN_HEIGHT - pipe[1])
                pipe_rect_top = pygame.Rect(pipe[0], 0, brique_img.get_width(), pipe[1] - pipe[3])
                if bird_rect.colliderect(pipe_rect) or bird_rect.colliderect(pipe_rect_top) or bird_y >= SCREEN_HEIGHT - 30 or bird_y <= 0:
                    running = False
                    break

            pygame.display.update()
            pygame.time.Clock().tick(30)

        if not game_over_screen(screen, score):
            break  # Quitter si l'utilisateur choisit de ne pas recommencer

        running = True  # Recommencer le jeu
        video_played = False  # Réinitialiser pour le prochain jeu

    pygame.quit()

if __name__ == "__main__":
    main()

In [20]:
"""
Jeu d'anniversaire : un jeu simple où l'objectif est de naviguer un oiseau à travers des tuyaux
et de débloquer une vidéo surprise en atteignant un score élevé.
"""

import pygame
import sys
import os
import random
import datetime
from moviepy import VideoFileClip  # Pour lire une vidéo
import numpy as np  # Utilisé pour les calculs dans certaines fonctions

# Constantes globales
WHITE = (255, 255, 255)  # Couleur blanche pour divers éléments
SCREEN_WIDTH, SCREEN_HEIGHT = 400, 600  # Taille de la fenêtre
BIRD_X = 50  # Position horizontale de l'oiseau
BIRD_Y = 300  # Position verticale initiale de l'oiseau
GRAVITY = 0.25  # Gravité appliquée à l'oiseau
FLAP_STRENGTH = -5  # Force appliquée lorsqu'on appuie sur les flèches
PIPE_SPEED_INIT = -4  # Vitesse initiale des tuyaux
PIPE_GAP_MIN = 100  # Écart minimal entre les tuyaux
PIPE_GAP_MAX = 180  # Écart maximal entre les tuyaux
FONT_NAME = pygame.font.match_font('arial')  # Police par défaut

# Fonction utilitaire : afficher du texte sur l'écran
def draw_text(surf, text, size, x, y, color=(255, 255, 255), font_name=FONT_NAME):
    """ Affiche un texte centré sur l'écran """
    font = pygame.font.Font(font_name, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    text_rect.midtop = (x, y)
    surf.blit(text_surface, text_rect)

# Fonction utilitaire : générer des flocons de neige
def create_snowflakes(num_flakes):
    """ Génère des positions aléatoires pour les flocons de neige """
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT),
             random.randint(2, 5)] for _ in range(num_flakes)]

def update_and_draw_snowflakes(screen, snowflakes):
    """ Met à jour et dessine les flocons de neige sur l'écran """
    for flake in snowflakes:
        flake[1] += flake[2]  # Faire descendre les flocons
        pygame.draw.circle(screen, WHITE, (flake[0], flake[1]), flake[2])
        if flake[1] > SCREEN_HEIGHT:  # Réinitialiser les flocons qui sortent de l'écran
            flake[0] = random.randint(0, SCREEN_WIDTH)
            flake[1] = -5
            flake[2] = random.randint(2, 5)

# Fonction utilitaire : dessiner un bouton cliquable
def draw_button(surf, text, x, y, width, height):
    """ Dessine un bouton interactif """
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    button_clicked = False

    # Vérifie si la souris est au-dessus du bouton
    if x + width > mouse[0] > x and y + height > mouse[1] > y:
        pygame.draw.rect(surf, (200, 200, 200), (x, y, width, height))  # Couleur plus claire
        if click[0] == 1:  # Si le bouton gauche est cliqué
            button_clicked = True
    else:
        pygame.draw.rect(surf, (170, 170, 170), (x, y, width, height))  # Couleur plus foncée

    draw_text(surf, text, 20, x + width / 2, y + 10)
    return button_clicked

# Écran de démarrage
def start_screen(screen):
    """ Affiche l'écran d'accueil avec des instructions """
    snowflakes = create_snowflakes(100)  # Générer les flocons de neige
    intro_text = [
        "Bienvenue au jeu d'anniversaire!",
        "Aidez le personnage à atteindre",
        "le score de 10 pour une surprise spéciale!",
        "",
        "Utilisez la flèche du haut pour monter,",
        "et celle du bas pour descendre.",
        "Bonne chance!"
    ]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))  # Couleur de fond
        update_and_draw_snowflakes(screen, snowflakes)

        # Afficher le titre et les instructions
        draw_text(screen, "Joyeux Anniversaire!", 40, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 10, color=(255, 50, 0))
        y_offset = 150
        for line in intro_text:
            draw_text(screen, line, 20, SCREEN_WIDTH / 2, y_offset)
            y_offset += 30

        # Bouton pour commencer
        if draw_button(screen, "Commencer", 150, 450, 100, 50):
            break

        pygame.display.flip()
        pygame.time.Clock().tick(15)

# Fonction pour dessiner les tuyaux
def draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img):
    """ Dessine les tuyaux (haut et bas) avec des briques """
    brique_height = brique_img.get_height()

    # Dessiner les tuyaux supérieurs
    y_top = pipe_y - pipe_gap - brique_height
    while y_top > -brique_height:
        screen.blit(pygame.transform.flip(brique_img, False, True), (pipe_x, y_top))
        y_top -= brique_height

    # Dessiner les tuyaux inférieurs
    y_bottom = pipe_y
    while y_bottom < SCREEN_HEIGHT:
        screen.blit(brique_img, (pipe_x, y_bottom))
        y_bottom += brique_height

# Fonction principale du jeu
def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("Joyeux Anniversaire!")

    # Charger les images
    bird_img = pygame.image.load('resources/mario_volant.png').convert_alpha()
    bird_img = pygame.transform.scale(bird_img, (50, 50))
    brique_img = pygame.image.load('resources/brique.png').convert_alpha()
    brique_img = pygame.transform.scale(brique_img, (50, 50))  # Ajuste à 50x50 pixels

    # Variables de jeu
    bird_y = BIRD_Y
    bird_velocity = 0
    pipes = [[300, random.randint(100, 400), False, random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)]]
    score = 0
    pipe_speed = PIPE_SPEED_INIT
    snowflakes = create_snowflakes(25)
    video_played = False
    last_speed_increase_score = 0  # Initialisation de la variable pour éviter l'erreur

    running = True
    while running:
        bird_rect = bird_img.get_rect(topleft=(BIRD_X, bird_y))  # Définir le rectangle de collision de l'oiseau

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    bird_velocity = FLAP_STRENGTH
                elif event.key == pygame.K_DOWN:
                    bird_velocity = -FLAP_STRENGTH

        bird_velocity += GRAVITY
        bird_y += bird_velocity

        # Mettre à jour l'écran
        screen.fill((15, 5, 107))
        update_and_draw_snowflakes(screen, snowflakes)
        screen.blit(bird_img, (BIRD_X, bird_y))

        # Gérer les tuyaux
        for pipe in pipes:
            pipe[0] += pipe_speed  # Déplacement horizontal
            draw_pipe(screen, pipe[0], pipe[1], pipe[3], brique_img)  # Dessiner le tuyau

            # Ajouter de nouveaux tuyaux
            if pipe[0] < -50:
                new_pipe_y = random.randint(100, 400)
                pipes.append([400, new_pipe_y, False, random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)])
                pipes.pop(0)

            # Vérifier si un tuyau est passé
            if pipe[0] + brique_img.get_width() < BIRD_X and not pipe[2]:
                score += 1
                pipe[2] = True

            # Vérifier les collisions
            pipe_rect = pygame.Rect(pipe[0], pipe[1], brique_img.get_width(), SCREEN_HEIGHT - pipe[1])
            pipe_rect_top = pygame.Rect(pipe[0], 0, brique_img.get_width(), pipe[1] - pipe[3])
            if bird_rect.colliderect(pipe_rect) or bird_rect.colliderect(pipe_rect_top):
                running = False

        # Condition pour jouer la vidéo
        if score >= 10 and not video_played:
            play_video(screen, 'resources/test_video.avi')
            video_played = True

        # Afficher le score
        draw_text(screen, f"Score: {score}", 24, SCREEN_WIDTH - 100, 50)

        # Vérifier si le score atteint le seuil pour augmenter la vitesse des tuyaux
        if score // SCORE_INTERVAL_FOR_SPEED_INCREASE > last_speed_increase_score:
            pipe_speed -= 1  # Augmenter la vitesse des tuyaux
            last_speed_increase_score = score // SCORE_INTERVAL_FOR_SPEED_INCREASE

        # Vérifier si l'oiseau touche les bords de l'écran
        if bird_y >= SCREEN_HEIGHT - 30 or bird_y <= 0:
            print("Collision avec le bord de l'écran!")
            running = False

        # Mettre à jour l'affichage
        pygame.display.update()
        pygame.time.Clock().tick(30)

    # Écran de fin de jeu
    if not game_over_screen(screen, score):
        return  # Quitter si le joueur ne veut pas rejouer

    # Relancer le jeu
    main()

    pygame.quit()

if __name__ == "__main__":
    main()

Collision avec le bord de l'écran!


In [21]:
import pygame
import sys
import os
import random
import datetime
import moviepy
from moviepy import VideoFileClip  # Importer moviepy pour lire la vidéo

# Hardcoded metadata values
app_name = "Joyeux Anniversaire!"

# Constantes
WHITE = (255, 255, 255)
SCREEN_WIDTH, SCREEN_HEIGHT = 400, 600
BIRD_X = 50
BIRD_Y = 300
GRAVITY = 0.25
FLAP_STRENGTH = -5
PIPE_SPEED_INIT = -4
PIPE_GAP = 150
PIPE_GAP_MIN = 100  # Valeur minimale de l'écart
PIPE_GAP_MAX = 180  # Valeur maximale de l'écart
SPEED_INCREASE_INTERVAL = 3000
SCORE_INTERVAL_FOR_SPEED_INCREASE = 5

FONT_NAME = pygame.font.match_font('arial')

def draw_text(surf, text, size, x, y, color=(255, 255, 255), font_name=FONT_NAME):
    font = pygame.font.Font(font_name, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    text_rect.midtop = (x, y)
    surf.blit(text_surface, text_rect)

def create_snowflakes(num_flakes):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT),
             random.randint(2, 5)] for _ in range(num_flakes)]

def update_and_draw_snowflakes(screen, snowflakes):
    for flake in snowflakes:
        flake[1] += flake[2]
        pygame.draw.circle(screen, WHITE, (flake[0], flake[1]), flake[2])
        if flake[1] > SCREEN_HEIGHT:
            flake[0] = random.randint(0, SCREEN_WIDTH)
            flake[1] = -5
            flake[2] = random.randint(2, 5)

def draw_button(surf, text, x, y, width, height):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    button_clicked = False

    if x + width > mouse[0] > x and y + height > mouse[1] > y:
        pygame.draw.rect(surf, (200, 200, 200), (x, y, width, height))
        if click[0] == 1:
            button_clicked = True
    else:
        pygame.draw.rect(surf, (170, 170, 170), (x, y, width, height))

    draw_text(surf, text, 20, x + width / 2, y + 10)
    return button_clicked

def start_screen(screen):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)
    intro_text = [
        "Bienvenue au jeu d'anniversaire!",
        "Aidez le personnage à atteindre",
        "le score de 10 pour une surprise spéciale!",
        "",
        "Utilisez la flèche du haut pour monter,",
        "et celle du bas pour descendre.",
        "Bonne chance!"
    ]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)

        draw_text(screen, "Joyeux Anniversaire!", 40, SCREEN_WIDTH / 2,
                  SCREEN_HEIGHT / 10, color=(255, 50, 0)) 
        y_offset = 150
        for line in intro_text:
            draw_text(screen, line, 20, SCREEN_WIDTH / 2, y_offset)
            y_offset += 30

        if draw_button(screen, "Commencer", 150, 450, 100, 50):
            break

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def game_over_screen(screen, final_score):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)  # Dessiner les flocons de neige

        draw_text(screen, "Game Over", 48, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 4)
        score_text = f"Score Final: {final_score}"
        draw_text(screen, score_text, 36, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50)

        if draw_button(screen, "Rejouer", 100, 350, 100, 50):
            return True  # Indique de recommencer le jeu
        if draw_button(screen, "Quitter", 200, 350, 100, 50):
            return False  # Indique de quitter le jeu

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def create_stars(num_stars):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT)] for _ in range(num_stars)]

def draw_stars(screen, stars):
    for star in stars:
        pygame.draw.circle(screen, (255, 255, 0), star, random.randint(1, 3))

def wait_for_key():
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                waiting = False

def play_video(screen, video_path):
    clip = VideoFileClip(video_path)
    clip = clip.resize(height=SCREEN_HEIGHT, width=SCREEN_WIDTH)
    clock = pygame.time.Clock()

    for frame in clip.iter_frames(fps=30, dtype="uint8"):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        frame_surface = pygame.surfarray.make_surface(frame.swapaxes(0, 1))
        screen.blit(frame_surface, (0, 0))
        pygame.display.update()
        clock.tick(30)
    clip.close()

def draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img, plante_img, has_plant):
    brique_height = brique_img.get_height()
    brique_width = brique_img.get_width()
    plante_height = plante_img.get_height()
    plante_width = plante_img.get_width()

    # Dessiner les briques pour la partie supérieure (tuyau inversé)
    y_top = pipe_y - pipe_gap - brique_height
    while y_top > -brique_height:
        screen.blit(pygame.transform.flip(brique_img, False, True), (pipe_x, y_top))
        y_top -= brique_height

    # Dessiner les briques pour la partie inférieure
    y_bottom = pipe_y
    while y_bottom < SCREEN_HEIGHT:
        screen.blit(brique_img, (pipe_x, y_bottom))
        y_bottom += brique_height

    # Dessiner la plante si le tuyau a une plante
    if has_plant:
        screen.blit(plante_img, (pipe_x + (brique_width - plante_width) // 2, pipe_y - plante_height))

def main():
    pygame.init()
    pygame.display.set_caption(app_name)
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    start_screen(screen)

    # Initialisation de la logique du jeu
    bird_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/mario_volant.png').convert_alpha()
    bird_img = pygame.transform.scale(bird_img, (50, 50))
    brique_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/brique.png').convert_alpha()
    brique_img = pygame.transform.scale(brique_img, (50, 50))  # Redimensionner à 50x50 si nécessaire
    plante_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/plante.png').convert_alpha()
    plante_img = pygame.transform.scale(plante_img, (50, 75))  # Ajustez la taille selon vos besoins

    stars = create_stars(10)  # Choisissez le nombre d'étoiles

    video_played = False
    last_speed_increase_score = 0

    running = True
    while running:
        bird_y = BIRD_Y
        bird_velocity = 0
        # Ajouter 'has_plant' au tuyau initial
        pipes = [[300, random.randint(100, 400), False, random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX), random.choice([True, False])]]
        score = 0
        pipe_speed = PIPE_SPEED_INIT
        snowflakes = create_snowflakes(25)

        while running:
            # Logique du jeu
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        bird_velocity = FLAP_STRENGTH
                    elif event.key == pygame.K_DOWN:
                        bird_velocity = -FLAP_STRENGTH

            bird_velocity += GRAVITY
            bird_y += bird_velocity

            screen.fill((15, 5, 107))
            draw_stars(screen, stars)  # Dessiner les étoiles
            update_and_draw_snowflakes(screen, snowflakes)
            screen.blit(bird_img, (BIRD_X, bird_y))

            # Gestion des tuyaux
            for pipe in pipes:
                pipe[0] += pipe_speed
                pipe_x, pipe_y, pipe_passed, pipe_gap, has_plant = pipe

                draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img, plante_img, has_plant)

                # Générer de nouveaux tuyaux
                if pipe[0] < -50:
                    new_gap = random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)  # Nouvel écart aléatoire
                    new_pipe_y = random.randint(100, 400)
                    has_plant = random.choice([True, False])  # Décider si le nouveau tuyau aura une plante
                    pipes.append([400, new_pipe_y, False, new_gap, has_plant])
                    pipes.pop(0)

                # Vérifier si le tuyau a été passé pour augmenter le score
                if pipe[0] + brique_img.get_width() < BIRD_X and not pipe_passed:
                    score += 1
                    pipe[2] = True  # Marquer le tuyau comme passé

            draw_text(screen, f"Score: {score}", 24, SCREEN_WIDTH - 100, 50)

            # Vérifier si le score a atteint un nouveau multiple de SCORE_INTERVAL_FOR_SPEED_INCREASE
            if score // SCORE_INTERVAL_FOR_SPEED_INCREASE > last_speed_increase_score:
                pipe_speed -= 1  # Augmenter la vitesse des tuyaux
                last_speed_increase_score = score // SCORE_INTERVAL_FOR_SPEED_INCREASE

            # Jouer la vidéo lorsque le score atteint 10
            if score >= 10 and not video_played:
                play_video(screen, '/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/test_video.avi')
                video_played = True  # Éviter de rejouer la vidéo

            bird_rect = bird_img.get_rect(topleft=(BIRD_X, bird_y))
            for pipe in pipes:
                pipe_x, pipe_y, pipe_passed, pipe_gap, has_plant = pipe
                pipe_width = brique_img.get_width()
                # Rectangle du tuyau inférieur
                pipe_rect = pygame.Rect(pipe_x, pipe_y, pipe_width, SCREEN_HEIGHT - pipe_y)
                # Rectangle du tuyau supérieur
                pipe_rect_top = pygame.Rect(pipe_x, 0, pipe_width, pipe_y - pipe_gap)
                # Vérifier la collision avec les tuyaux
                if bird_rect.colliderect(pipe_rect) or bird_rect.colliderect(pipe_rect_top):
                    running = False
                    break
                # Vérifier la collision avec la plante si elle est présente
                if has_plant:
                    plante_rect = plante_img.get_rect()
                    plante_rect.x = pipe_x + (pipe_width - plante_img.get_width()) // 2
                    plante_rect.y = pipe_y - plante_img.get_height()
                    if bird_rect.colliderect(plante_rect):
                        running = False
                        break
            # Vérifier si l'oiseau est sorti de l'écran
            if bird_y >= SCREEN_HEIGHT - 30 or bird_y <= 0:
                running = False

            pygame.display.update()
            pygame.time.Clock().tick(30)

        if not game_over_screen(screen, score):
            break  # Quitter si l'utilisateur choisit de ne pas recommencer

        running = True  # Recommencer le jeu
        video_played = False  # Réinitialiser pour le prochain jeu

    pygame.quit()

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [9]:
import pygame
import sys
import os
import random
import datetime
import moviepy
from moviepy import VideoFileClip  # Importer moviepy pour lire la vidéo

# Hardcoded metadata values
app_name = "Joyeux Anniversaire!"

# Constantes
WHITE = (255, 255, 255)
SCREEN_WIDTH, SCREEN_HEIGHT = 400, 600
BIRD_X = 50
BIRD_Y = 300
GRAVITY = 0.25
FLAP_STRENGTH = -5
PIPE_SPEED_INIT = -4
PIPE_GAP = 150
PIPE_GAP_MIN = 100  # Valeur minimale de l'écart
PIPE_GAP_MAX = 180  # Valeur maximale de l'écart
SPEED_INCREASE_INTERVAL = 3000
SCORE_INTERVAL_FOR_SPEED_INCREASE = 5

FONT_NAME = pygame.font.match_font('arial')

def draw_text(surf, text, size, x, y, color=(255, 255, 255), font_name=FONT_NAME):
    font = pygame.font.Font(font_name, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    text_rect.midtop = (x, y)
    surf.blit(text_surface, text_rect)

def create_snowflakes(num_flakes):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT),
             random.randint(2, 5)] for _ in range(num_flakes)]

def update_and_draw_snowflakes(screen, snowflakes):
    for flake in snowflakes:
        flake[1] += flake[2]
        pygame.draw.circle(screen, WHITE, (flake[0], flake[1]), flake[2])
        if flake[1] > SCREEN_HEIGHT:
            flake[0] = random.randint(0, SCREEN_WIDTH)
            flake[1] = -5
            flake[2] = random.randint(2, 5)

def draw_button(surf, text, x, y, width, height):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    button_clicked = False

    if x + width > mouse[0] > x and y + height > mouse[1] > y:
        pygame.draw.rect(surf, (200, 200, 200), (x, y, width, height))
        if click[0] == 1:
            button_clicked = True
    else:
        pygame.draw.rect(surf, (170, 170, 170), (x, y, width, height))

    draw_text(surf, text, 20, x + width / 2, y + 10)
    return button_clicked

def start_screen(screen):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)
    intro_text = [
        "Bienvenue au jeu d'anniversaire!",
        "Aidez le personnage à atteindre",
        "le score de 10 pour une surprise spéciale!",
        "",
        "Utilisez la flèche du haut pour monter,",
        "et celle du bas pour descendre.",
        "Bonne chance!"
    ]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)

        draw_text(screen, "Joyeux Anniversaire!", 40, SCREEN_WIDTH / 2,
                  SCREEN_HEIGHT / 10, color=(255, 50, 0)) 
        y_offset = 150
        for line in intro_text:
            draw_text(screen, line, 20, SCREEN_WIDTH / 2, y_offset)
            y_offset += 30

        if draw_button(screen, "Commencer", 150, 450, 100, 50):
            break

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def game_over_screen(screen, final_score):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)  # Dessiner les flocons de neige

        draw_text(screen, "Game Over", 48, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 4)
        score_text = f"Score Final: {final_score}"
        draw_text(screen, score_text, 36, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50)

        if draw_button(screen, "Rejouer", 100, 350, 100, 50):
            return True  # Indique de recommencer le jeu
        if draw_button(screen, "Quitter", 200, 350, 100, 50):
            return False  # Indique de quitter le jeu

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def create_stars(num_stars):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT)] for _ in range(num_stars)]

def draw_stars(screen, stars):
    for star in stars:
        pygame.draw.circle(screen, (255, 255, 0), star, random.randint(1, 3))

def wait_for_key():
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                waiting = False

def play_video(screen, video_path):
    clip = VideoFileClip(video_path)
    clip = clip.resize(height=SCREEN_HEIGHT, width=SCREEN_WIDTH)
    clock = pygame.time.Clock()

    for frame in clip.iter_frames(fps=30, dtype="uint8"):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        frame_surface = pygame.surfarray.make_surface(frame.swapaxes(0, 1))
        screen.blit(frame_surface, (0, 0))
        pygame.display.update()
        clock.tick(30)
    clip.close()

def draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img, plante_img, has_plant):
    brique_height = brique_img.get_height()
    brique_width = brique_img.get_width()
    plante_height = plante_img.get_height()
    plante_width = plante_img.get_width()

    # Dessiner les briques pour la partie supérieure (tuyau inversé)
    y_top = pipe_y - pipe_gap - brique_height
    while y_top > -brique_height:
        screen.blit(pygame.transform.flip(brique_img, False, True), (pipe_x, y_top))
        y_top -= brique_height

    # Dessiner les briques pour la partie inférieure
    y_bottom = pipe_y
    while y_bottom < SCREEN_HEIGHT:
        screen.blit(brique_img, (pipe_x, y_bottom))
        y_bottom += brique_height

    # Dessiner la plante si le tuyau a une plante
    if has_plant:
        screen.blit(plante_img, (pipe_x + (brique_width - plante_width) // 2, pipe_y - plante_height))

def main():
    pygame.init()
    pygame.display.set_caption(app_name)
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    start_screen(screen)

    # Initialisation de la logique du jeu
    bird_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/mario_volant.png').convert_alpha()
    bird_img = pygame.transform.scale(bird_img, (50, 50))
    brique_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/brique.png').convert_alpha()
    brique_img = pygame.transform.scale(brique_img, (50, 50))  # Redimensionner à 50x50 si nécessaire
    plante_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/plante.png').convert_alpha()
    plante_img = pygame.transform.scale(plante_img, (50, 75))  # Ajustez la taille selon vos besoins

    stars = create_stars(10)  # Choisissez le nombre d'étoiles

    video_played = False
    last_speed_increase_score = 0

    running = True
    while running:
        bird_y = BIRD_Y
        bird_velocity = 0
        # Ajouter 'has_plant' au tuyau initial
        pipes = [[300, random.randint(100, 400), False, random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX), random.choice([True, False])]]
        score = 0
        pipe_speed = PIPE_SPEED_INIT
        snowflakes = create_snowflakes(25)

        while running:
            # Logique du jeu
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        bird_velocity = FLAP_STRENGTH
                    elif event.key == pygame.K_DOWN:
                        bird_velocity = -FLAP_STRENGTH

            bird_velocity += GRAVITY
            bird_y += bird_velocity

            screen.fill((15, 5, 107))
            draw_stars(screen, stars)  # Dessiner les étoiles
            update_and_draw_snowflakes(screen, snowflakes)
            screen.blit(bird_img, (BIRD_X, bird_y))

            # Gestion des tuyaux
            for pipe in pipes:
                pipe[0] += pipe_speed
                pipe_x, pipe_y, pipe_passed, pipe_gap, has_plant = pipe

                draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img, plante_img, has_plant)

                # Générer de nouveaux tuyaux
                if pipe[0] < -50:
                    new_gap = random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)  # Nouvel écart aléatoire
                    new_pipe_y = random.randint(100, 400)
                    has_plant = random.choice([True, False])  # Décider si le nouveau tuyau aura une plante
                    pipes.append([400, new_pipe_y, False, new_gap, has_plant])
                    pipes.pop(0)

                # Vérifier si le tuyau a été passé pour augmenter le score
                if pipe[0] + brique_img.get_width() < BIRD_X and not pipe_passed:
                    score += 1
                    pipe[2] = True  # Marquer le tuyau comme passé

            draw_text(screen, f"Score: {score}", 24, SCREEN_WIDTH - 100, 50)

            # Vérifier si le score a atteint un nouveau multiple de SCORE_INTERVAL_FOR_SPEED_INCREASE
            if score // SCORE_INTERVAL_FOR_SPEED_INCREASE > last_speed_increase_score:
                pipe_speed -= 1  # Augmenter la vitesse des tuyaux
                last_speed_increase_score = score // SCORE_INTERVAL_FOR_SPEED_INCREASE

            # Jouer la vidéo lorsque le score atteint 10
            if score >= 10 and not video_played:
                play_video(screen, '/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/test_video.avi')
                video_played = True  # Éviter de rejouer la vidéo

            bird_rect = bird_img.get_rect(topleft=(BIRD_X, bird_y))
            for pipe in pipes:
                pipe_x, pipe_y, pipe_passed, pipe_gap, has_plant = pipe
                pipe_width = brique_img.get_width()
                # Rectangle du tuyau inférieur
                pipe_rect = pygame.Rect(pipe_x, pipe_y, pipe_width, SCREEN_HEIGHT - pipe_y)
                # Rectangle du tuyau supérieur
                pipe_rect_top = pygame.Rect(pipe_x, 0, pipe_width, pipe_y - pipe_gap)
                # Vérifier la collision avec les tuyaux
                if bird_rect.colliderect(pipe_rect) or bird_rect.colliderect(pipe_rect_top):
                    running = False
                    break
                # Vérifier la collision avec la plante si elle est présente
                if has_plant:
                    plante_rect = plante_img.get_rect()
                    plante_rect.x = pipe_x + (pipe_width - plante_img.get_width()) // 2
                    plante_rect.y = pipe_y - plante_img.get_height()
                    if bird_rect.colliderect(plante_rect):
                        running = False
                        break
            # Vérifier si l'oiseau est sorti de l'écran
            if bird_y >= SCREEN_HEIGHT - 30 or bird_y <= 0:
                running = False

            pygame.display.update()
            pygame.time.Clock().tick(30)

        if not game_over_screen(screen, score):
            break  # Quitter si l'utilisateur choisit de ne pas recommencer

        running = True  # Recommencer le jeu
        video_played = False  # Réinitialiser pour le prochain jeu

    pygame.quit()

if __name__ == "__main__":
    main()


SystemExit: 

In [10]:
import pygame
import sys
import os
import random
import datetime
import moviepy
from moviepy import VideoFileClip  # Importer moviepy pour lire la vidéo

# Hardcoded metadata values
app_name = "Joyeux Anniversaire!"

# Constantes
WHITE = (255, 255, 255)
SCREEN_WIDTH, SCREEN_HEIGHT = 400, 600
BIRD_X = 50
BIRD_Y = 300
GRAVITY = 0.25
FLAP_STRENGTH = -5
PIPE_SPEED_INIT = -4
PIPE_GAP = 150
PIPE_GAP_MIN = 100  # Valeur minimale de l'écart
PIPE_GAP_MAX = 180  # Valeur maximale de l'écart
SPEED_INCREASE_INTERVAL = 3000
SCORE_INTERVAL_FOR_SPEED_INCREASE = 5

FONT_NAME = pygame.font.match_font('arial')

def draw_text(surf, text, size, x, y, color=(255, 255, 255), font_name=FONT_NAME):
    font = pygame.font.Font(font_name, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    text_rect.midtop = (x, y)
    surf.blit(text_surface, text_rect)

def create_snowflakes(num_flakes):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT),
             random.randint(2, 5)] for _ in range(num_flakes)]

def update_and_draw_snowflakes(screen, snowflakes):
    for flake in snowflakes:
        flake[1] += flake[2]
        pygame.draw.circle(screen, WHITE, (flake[0], flake[1]), flake[2])
        if flake[1] > SCREEN_HEIGHT:
            flake[0] = random.randint(0, SCREEN_WIDTH)
            flake[1] = -5
            flake[2] = random.randint(2, 5)

def draw_button(surf, text, x, y, width, height):
    mouse = pygame.mouse.get_pos()
    click = pygame.mouse.get_pressed()
    button_clicked = False

    if x + width > mouse[0] > x and y + height > mouse[1] > y:
        pygame.draw.rect(surf, (200, 200, 200), (x, y, width, height))
        if click[0] == 1:
            button_clicked = True
    else:
        pygame.draw.rect(surf, (170, 170, 170), (x, y, width, height))

    draw_text(surf, text, 20, x + width / 2, y + 10)
    return button_clicked

def start_screen(screen):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)
    intro_text = [
        "Bienvenue au jeu d'anniversaire!",
        "Aidez le personnage à atteindre",
        "le score de 10 pour une surprise spéciale!",
        "",
        "Utilisez la flèche du haut pour monter,",
        "et celle du bas pour descendre.",
        "Bonne chance!"
    ]

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)

        draw_text(screen, "Joyeux Anniversaire!", 40, SCREEN_WIDTH / 2,
                  SCREEN_HEIGHT / 10, color=(255, 50, 0))
        y_offset = 150
        for line in intro_text:
            draw_text(screen, line, 20, SCREEN_WIDTH / 2, y_offset)
            y_offset += 30

        if draw_button(screen, "Commencer", 150, 450, 100, 50):
            break

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def game_over_screen(screen, final_score):
    stars = create_stars(25)  # Création des étoiles
    snowflakes = create_snowflakes(100)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.fill((15, 5, 107))
        draw_stars(screen, stars)  # Dessiner les étoiles
        update_and_draw_snowflakes(screen, snowflakes)  # Dessiner les flocons de neige

        draw_text(screen, "Game Over", 48, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 4)
        score_text = f"Score Final: {final_score}"
        draw_text(screen, score_text, 36, SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 + 50)

        if draw_button(screen, "Rejouer", 100, 350, 100, 50):
            return True  # Indique de recommencer le jeu
        if draw_button(screen, "Quitter", 200, 350, 100, 50):
            return False  # Indique de quitter le jeu

        pygame.display.flip()
        pygame.time.Clock().tick(15)

def create_stars(num_stars):
    return [[random.randint(0, SCREEN_WIDTH),
             random.randint(0, SCREEN_HEIGHT)] for _ in range(num_stars)]

def draw_stars(screen, stars):
    for star in stars:
        pygame.draw.circle(screen, (255, 255, 0), star, random.randint(1, 3))

def wait_for_key():
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                waiting = False

def play_video(screen, video_path):
    clip = VideoFileClip(video_path)
    clip = clip.resize(height=SCREEN_HEIGHT, width=SCREEN_WIDTH)
    clock = pygame.time.Clock()

    for frame in clip.iter_frames(fps=30, dtype="uint8"):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        frame_surface = pygame.surfarray.make_surface(frame.swapaxes(0, 1))
        screen.blit(frame_surface, (0, 0))
        pygame.display.update()
        clock.tick(30)
    clip.close()

def draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img, plante_img, has_plant):
    brique_height = brique_img.get_height()
    brique_width = brique_img.get_width()
    plante_height = plante_img.get_height()
    plante_width = plante_img.get_width()

    # Dessiner les briques pour la partie supérieure (tuyau inversé)
    y_top = pipe_y - pipe_gap - brique_height
    while y_top > -brique_height:
        screen.blit(pygame.transform.flip(brique_img, False, True), (pipe_x, y_top))
        y_top -= brique_height

    # Dessiner les briques pour la partie inférieure
    y_bottom = pipe_y
    max_y_bottom = SCREEN_HEIGHT - 2 * brique_height if has_plant else SCREEN_HEIGHT  # Retirer deux briques si la plante est présente
    while y_bottom < max_y_bottom:
        screen.blit(brique_img, (pipe_x, y_bottom))
        y_bottom += brique_height

    # Dessiner la plante si le tuyau a une plante
    if has_plant:
        plant_position_y = max_y_bottom - plante_height
        screen.blit(plante_img, (pipe_x + (brique_width - plante_width) // 2, plant_position_y))

def main():
    pygame.init()
    pygame.display.set_caption(app_name)
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    start_screen(screen)

    # Initialisation de la logique du jeu
    bird_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/mario_volant.png').convert_alpha()
    bird_img = pygame.transform.scale(bird_img, (50, 50))
    brique_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/brique.png').convert_alpha()
    brique_img = pygame.transform.scale(brique_img, (50, 50))  # Redimensionner à 50x50 si nécessaire
    plante_img = pygame.image.load('/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/plante.png').convert_alpha()
    plante_img = pygame.transform.scale(plante_img, (50, 75))  # Ajustez la taille selon vos besoins

    stars = create_stars(10)  # Choisissez le nombre d'étoiles

    video_played = False
    last_speed_increase_score = 0

    running = True
    while running:
        bird_y = BIRD_Y
        bird_velocity = 0
        # Ajouter 'has_plant' au tuyau initial
        pipes = [[300, random.randint(100, 400), False, random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX), random.choice([True, False])]]
        score = 0
        pipe_speed = PIPE_SPEED_INIT
        snowflakes = create_snowflakes(25)

        while running:
            # Logique du jeu
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        bird_velocity = FLAP_STRENGTH
                    elif event.key == pygame.K_DOWN:
                        bird_velocity = -FLAP_STRENGTH

            bird_velocity += GRAVITY
            bird_y += bird_velocity

            screen.fill((15, 5, 107))
            draw_stars(screen, stars)  # Dessiner les étoiles
            update_and_draw_snowflakes(screen, snowflakes)
            screen.blit(bird_img, (BIRD_X, bird_y))

            # Gestion des tuyaux
            for pipe in pipes:
                pipe[0] += pipe_speed
                pipe_x, pipe_y, pipe_passed, pipe_gap, has_plant = pipe

                draw_pipe(screen, pipe_x, pipe_y, pipe_gap, brique_img, plante_img, has_plant)

                # Générer de nouveaux tuyaux
                if pipe[0] < -50:
                    new_gap = random.randint(PIPE_GAP_MIN, PIPE_GAP_MAX)  # Nouvel écart aléatoire
                    new_pipe_y = random.randint(100, 400)
                    has_plant = random.choice([True, False])  # Décider si le nouveau tuyau aura une plante
                    pipes.append([400, new_pipe_y, False, new_gap, has_plant])
                    pipes.pop(0)

                # Vérifier si le tuyau a été passé pour augmenter le score
                if pipe[0] + brique_img.get_width() < BIRD_X and not pipe_passed:
                    score += 1
                    pipe[2] = True  # Marquer le tuyau comme passé

            draw_text(screen, f"Score: {score}", 24, SCREEN_WIDTH - 100, 50)

            # Vérifier si le score a atteint un nouveau multiple de SCORE_INTERVAL_FOR_SPEED_INCREASE
            if score // SCORE_INTERVAL_FOR_SPEED_INCREASE > last_speed_increase_score:
                pipe_speed -= 1  # Augmenter la vitesse des tuyaux
                last_speed_increase_score = score // SCORE_INTERVAL_FOR_SPEED_INCREASE

            # Jouer la vidéo lorsque le score atteint 10
            if score >= 10 and not video_played:
                play_video(screen, '/Users/souchaud/Documents/Autre/2024_11_26_anniv_anatole/resources/test_video.avi')
                video_played = True  # Éviter de rejouer la vidéo

            bird_rect = bird_img.get_rect(topleft=(BIRD_X, bird_y))
            for pipe in pipes:
                pipe_x, pipe_y, pipe_passed, pipe_gap, has_plant = pipe
                pipe_width = brique_img.get_width()
                brique_height = brique_img.get_height()

                # Ajuster la hauteur du rectangle de collision du tuyau inférieur
                pipe_rect_height = SCREEN_HEIGHT - pipe_y - 2 * brique_height if has_plant else SCREEN_HEIGHT - pipe_y
                pipe_rect = pygame.Rect(pipe_x, pipe_y, pipe_width, pipe_rect_height)

                # Rectangle du tuyau supérieur
                pipe_rect_top = pygame.Rect(pipe_x, 0, pipe_width, pipe_y - pipe_gap)

                # Vérifier la collision avec les tuyaux
                if bird_rect.colliderect(pipe_rect) or bird_rect.colliderect(pipe_rect_top):
                    running = False
                    break

                # Vérifier la collision avec la plante si elle est présente
                if has_plant:
                    plante_rect = plante_img.get_rect()
                    plante_rect.x = pipe_x + (pipe_width - plante_img.get_width()) // 2
                    plante_rect.y = pipe_y - plante_img.get_height()
                    if bird_rect.colliderect(plante_rect):
                        running = False
                        break
            # Vérifier si l'oiseau est sorti de l'écran
            if bird_y >= SCREEN_HEIGHT - 30 or bird_y <= 0:
                running = False

            pygame.display.update()
            pygame.time.Clock().tick(30)

        if not game_over_screen(screen, score):
            break  # Quitter si l'utilisateur choisit de ne pas recommencer

        running = True  # Recommencer le jeu
        video_played = False  # Réinitialiser pour le prochain jeu

    pygame.quit()

if __name__ == "__main__":
    main()


SystemExit: 

toujours pas au bon endroit mais au moins elles sont la. 